In [1]:
%%capture
!pip install opentrons

# PCR script for the OT-2

### Labware Setup:

- Loads three 96-well plates: one for forward primers, one for reverse primers, and one as the destination plate.
- Loads a 15 mL Falcon tube rack to hold the mastermix.
- Loads one tip rack for the P300 pipette and two tip racks for the P10 pipette.


### Pipette Setup:

- Initializes a P300 single-channel pipette with a 300 µL tip rack.
- Initializes a P10 single-channel pipette with two 10 µL tip racks.

### Mastermix Transfer:

- Picks up a single tip with the P300 pipette.
- Transfers 92 µL of mastermix from the Falcon tube to each well in the destination plate using the same tip.
- Drops the tip after all wells have been filled with mastermix.


### Primer Transfer:

- For each well from A1 to G12:
- Picks up a new tip with the P10 pipette.
- Transfers 4 µL of the corresponding forward primer to the same well in the destination plate.
- Drops the tip.
- Picks up a new tip with the P10 pipette.
- Transfers 4 µL of the corresponding reverse primer to the same well in the destination plate.
- Drops the tip.


### Simulation:

Simulates the protocol execution, printing out the steps the robot would take.

In [2]:
from opentrons import simulate, protocol_api

# metadata
metadata = {
    'protocolName': 'PCR Setup',
    'author': 'Your Name <your.email@example.com>',
    'description': 'Add mastermix, forward primers, and reverse primers to destination plate',
    'apiLevel': '2.11'
}

def run(protocol: protocol_api.ProtocolContext):

    # labware
    forward_plate = protocol.load_labware('corning_96_wellplate_360ul_flat', '1')
    reverse_plate = protocol.load_labware('corning_96_wellplate_360ul_flat', '2')
    destination_plate = protocol.load_labware('corning_96_wellplate_360ul_flat', '3')
    falcon_rack = protocol.load_labware('opentrons_15_tuberack_falcon_15ml_conical', '4')
    mastermix_tube = falcon_rack.wells_by_name()['A1']

    # tip racks
    tiprack_300 = protocol.load_labware('opentrons_96_tiprack_300ul', '5')
    tiprack_10_1 = protocol.load_labware('opentrons_96_tiprack_10ul', '6')
    tiprack_10_2 = protocol.load_labware('opentrons_96_tiprack_10ul', '7')

    # pipettes
    p300 = protocol.load_instrument('p300_single_gen2', 'right', tip_racks=[tiprack_300])
    p10 = protocol.load_instrument('p10_single', 'left', tip_racks=[tiprack_10_1, tiprack_10_2])

    # list of well names in 96-well format from A1 to G12 - actually we can also do it to H12.
    wells_96 = [f'{row}{col}' for row in 'ABCDEFGH' for col in range(1, 13)]

    # transfer 92µl of mastermix to each well that will receive primers
    p300.pick_up_tip()
    for well in wells_96[:84]:  # from A1 to G12 (inclusive)
        p300.transfer(92, mastermix_tube, destination_plate[well], new_tip='never')
    p300.drop_tip()

    # transfer 4µl of forward and reverse primers to the destination plate
    for well in wells_96[:84]:  # from A1 to G12 (inclusive)
        p10.transfer(4, forward_plate[well], destination_plate[well], new_tip='always')
        p10.transfer(4, reverse_plate[well], destination_plate[well], new_tip='always')

# simulate the protocol
protocol_context = simulate.get_protocol_api('2.11')
run(protocol_context)

# print the protocol commands
for command in protocol_context.commands():
    print(command)


/Users/lucaslevassor/.opentrons/robot_settings.json not found. Loading defaults
Deck calibration not found.
/Users/lucaslevassor/.opentrons/deck_calibration.json not found. Loading defaults


Picking up tip from A1 of Opentrons OT-2 96 Tip Rack 300 µL on 5
Transferring 92.0 from A1 of Opentrons 15 Tube Rack with Falcon 15 mL Conical on 4 to A1 of Corning 96 Well Plate 360 µL Flat on 3
Aspirating 92.0 uL from A1 of Opentrons 15 Tube Rack with Falcon 15 mL Conical on 4 at 92.86 uL/sec
Dispensing 92.0 uL into A1 of Corning 96 Well Plate 360 µL Flat on 3 at 92.86 uL/sec
Transferring 92.0 from A1 of Opentrons 15 Tube Rack with Falcon 15 mL Conical on 4 to A2 of Corning 96 Well Plate 360 µL Flat on 3
Aspirating 92.0 uL from A1 of Opentrons 15 Tube Rack with Falcon 15 mL Conical on 4 at 92.86 uL/sec
Dispensing 92.0 uL into A2 of Corning 96 Well Plate 360 µL Flat on 3 at 92.86 uL/sec
Transferring 92.0 from A1 of Opentrons 15 Tube Rack with Falcon 15 mL Conical on 4 to A3 of Corning 96 Well Plate 360 µL Flat on 3
Aspirating 92.0 uL from A1 of Opentrons 15 Tube Rack with Falcon 15 mL Conical on 4 at 92.86 uL/sec
Dispensing 92.0 uL into A3 of Corning 96 Well Plate 360 µL Flat on 3 at 